<a href="https://colab.research.google.com/github/FlyAIBox/Agent_In_Action/blob/main/01-agent-llm-mcp/ASimpleAgentFramework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAME框架：AI智能体设计架构

本笔记实现了一个“可复用的智能体（Agent）最小框架”，用来演示如何基于 GAME 设计法将智能体拆分为四个核心部件，并以可插拔方式组织：

- **G（Goals / Instructions）**：目标与指令。描述智能体要实现的结果，以及实现策略/规则。
- **A（Actions）**：动作/工具。定义智能体可以调用的能力（如读取文件、列目录、结束会话等）。
- **M（Memory）**：记忆。跨回合保留上下文（用户输入、助手决策、工具执行结果），支持后续回合继续推理。
- **E（Environment）**：环境。动作在真实世界中的执行载体，负责真正“落地执行”动作并返回结果（含时间戳与错误信息）。

本框架通过一个统一的 **Agent** 循环（Loop）把 G/A/M/E 串起来：
1. 构造 Prompt（包含 Goals、可用 Actions 的函数调用Schema、Memory 历史）。
2. 发送给 LLM，得到“选择的动作以及参数”（函数调用）。」
3. 在 **Environment** 中执行该动作，得到结果（或错误）。
4. 将决策与结果写入 **Memory**，进入下一轮。
5. 如果动作为终止类动作（如 `terminate`），则结束循环。

你可以把 **Actions** 看成“能力接口”，把 **Environment** 看成“执行实现”。这种解耦使得：
- 你可以替换不同环境（本地、云端、GitHub Actions、容器等），而无需修改智能体决策逻辑；
- 你可以更换一组 Actions（比如从文件工具换成 Web API 工具），而无需修改主循环；
- 你可以替换/扩展 **AgentLanguage**（Prompt 格式与解析逻辑），以适配“函数调用/纯文本解析”等不同LLM交互方式。

本笔记下半部分提供了一个最小示例：
- 定义了 3 个动作：`list_project_files`、`read_project_file`、`terminate`
- 目标：读取项目文件并在结束时输出 README 内容（示例运行环境为空目录时会直接终止）
- 使用 OpenAI 调用 `gpt-4o`，但可轻松替换为任意 LLM 提供商

通过阅读与运行本笔记，你将能掌握：
- 如何将智能体设计（GAME）直接映射为代码结构；
- 如何注册工具、格式化 Prompt、解析 LLM 工具调用并在环境中执行；
- 如何使用记忆把“决策 + 结果”闭环起来，形成稳健的 Agent Loop。


In [1]:
# 安装必要的依赖包
!!pip install openai==1.107.0

['Collecting openai==1.107.0',
 '  Downloading openai-1.107.0-py3-none-any.whl.metadata (29 kB)',
 'Requirement already satisfied: anyio<5,>=3.5.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (4.11.0)',
 'Requirement already satisfied: distro<2,>=1.7.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (1.9.0)',
 'Requirement already satisfied: httpx<1,>=0.23.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (0.28.1)',
 'Requirement already satisfied: jiter<1,>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (0.11.1)',
 'Requirement already satisfied: pydantic<3,>=1.9.0 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (2.11.10)',
 'Requirement already satisfied: sniffio in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (1.3.1)',
 'Requirement already satisfied: tqdm>4 in /usr/local/lib/python3.12/dist-packages (from openai==1.107.0) (4.67.1)',
 'Requirement already satisfied: 

In [2]:
# 导入必要的模块
import os, getpass

def _set_env(var: str):
    """
    设置环境变量的辅助函数

    参数:
        var (str): 要设置的环境变量名称

    功能:
        - 检查环境变量是否已存在
        - 如果不存在，则提示用户输入并设置
    """
    if not os.environ.get(var):  # 检查环境变量是否已设置
        os.environ[var] = getpass.getpass(f"{var}: ")  # 安全地获取用户输入

# 设置 OpenAI API 密钥
# 这是使用 OpenAI 模型所必需的
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")

OPENAI_API_KEY: ··········
OPENAI_BASE_URL: ··········


In [ ]:
# =============================== 核心框架：导入与类型定义 ===============================
# 说明：以下代码实现了一个最小可复用的智能体框架（面向函数调用工具）。
# - 不修改任何原有逻辑，仅通过中文注释解释设计意图与用法。
# - 关键模块：Prompt 数据结构、LLM 响应函数、Goal/Action/ActionRegistry、Memory、Environment、AgentLanguage、Agent。

import json
import time
import traceback
from openai import OpenAI # 用于调用OpenAI API
from dataclasses import dataclass, field
from typing import List, Callable, Dict, Any


# 大语言模型
client=OpenAI(
    base_url=os.environ['OPENAI_BASE_URL'],
    api_key=os.environ['OPENAI_API_KEY']
)

# Prompt：封装要发给 LLM 的消息与工具定义
# - messages：对话上下文（系统/用户/助手三类）
# - tools：工具（函数）调用的 JSON Schema 描述（让 LLM 能“看见”可用的动作）
# - metadata：元数据（可选扩展，用 dict 保存）
@dataclass
class Prompt:
    messages: List[Dict] = field(default_factory=list)
    tools: List[Dict] = field(default_factory=list)
    metadata: dict = field(default_factory=dict)  


# generate_response：统一的 LLM 调用入口
# - 入参是 Prompt，内部自动根据是否提供 tools 来决定是否启用函数调用能力
# - 目标：把模型提供商与主循环解耦；将来切换模型时无需改 Agent 逻辑
# - 返回：
#   * 无工具时：直接返回助手文本
#   * 有工具时：优先解析 tool_calls（并转为 {tool, args} 的 JSON 字符串）
#               若无工具调用，则退化为普通文本回复
def generate_response(prompt: Prompt) -> str:
    """调用大语言模型（LLM）生成响应：
    - 当未提供 tools（函数调用能力）时，作为普通对话返回文本
    - 当提供 tools 时，优先解析函数调用的结构化结果；若无函数调用则退化为普通文本
    """

    messages = prompt.messages
    tools = prompt.tools

    result = None

    if not tools:
        # 无工具：普通对话
        response = client.chat.completions.create(
            model="gpt-4o",   # 指定使用的模型
            messages=messages,  # 发送消息历史
            max_tokens=1024   # 限制响应长度
        )
        result = response.choices[0].message.content
    else:
        # 有工具：提示模型按函数调用格式返回 tool_calls
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools,
            max_tokens=1024
        )

        if response.choices[0].message.tool_calls:
            # 这里仅取第一个工具调用作为最小可运行演示
            tool = response.choices[0].message.tool_calls[0]
            result = {
                "tool": tool.function.name,
                "args": json.loads(tool.function.arguments),
            }
            # 将 dict 序列化为字符串，便于统一处理与存入记忆
            result = json.dumps(result)
        else:
            # 即使提供了 tools，也可能返回纯文本（例如模型策略判断不调用工具）
            result = response.choices[0].message.content


    return result


# Goal：目标对象
# - priority：目标优先级（便于排序/裁剪）
# - name/description：目标名称和详细说明（同时涵盖“要做什么/如何做”）
# 使用 @dataclass 装饰器定义 Goal 为一个不可变的数据类（frozen=True），这意味着其实例一旦创建，其属性值就不能被修改，有助于保证数据安全和可靠性。
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str


# Action：动作/工具的抽象
# - name：动作名（作为工具名暴露给 LLM）
# - function：实际执行的 Python 函数
# - description：工具说明，帮助 LLM 选择正确工具
# - parameters：JSON Schema（决定 LLM 该如何拼好参数）
# - terminal：是否为“终止型”动作（被选中后终止主循环）
class Action:
    def __init__(self,
                 name: str,
                 function: Callable,
                 description: str,
                 parameters: Dict,
                 terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        """执行该动作所绑定的底层函数，参数通过关键字形式解包传入"""
        # 解包参数并调用底层实现函数
        return self.function(**args)


# ActionRegistry：动作/工具注册表
# - 负责集中管理动作/工具对象，支持按名称检索与批量导出供 AgentLanguage 生成工具Schema
class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str) -> [Action, None]:
        return self.actions.get(name, None)

    def get_actions(self) -> List[Action]:
        """获取所有已注册的动作，按注册顺序返回列表"""
        return list(self.actions.values())


# Memory：回合记忆
# - items：统一存储“用户/助手/环境”等事件，形成对话历史
# - 通过 get_memories 提供最近N条消息给提示构造使用
# - 通过 copy_without_system_memories 可过滤掉系统消息（某些场景需要）
class Memory:
    def __init__(self):
        self.items = []  # Basic conversation histor

    def add_memory(self, memory: dict):
        """将一条记忆事件追加到工作记忆，用于后续提示词构造与推理"""
        self.items.append(memory)

    def get_memories(self, limit: int = None) -> List[Dict]:
        """获取用于提示词的对话历史；可通过 limit 限制条数以控制上下文长度"""
        return self.items[:limit]

    def copy_without_system_memories(self):
        """返回一份不包含系统类型（type==system）记忆的副本，用于部分提示场景"""
        filtered_items = [m for m in self.items if m["type"] != "system"]
        memory = Memory()
        memory.items = filtered_items
        return memory


# Environment：环境层（动作的真实执行者）
# - execute_action：捕获执行异常，统一返回结构（是否执行成功/错误/traceback/时间戳）
# - format_result：为成功结果补充元数据（时间戳），便于记录与日志化
class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        """执行指定动作并返回标准化结果；捕获异常并提供错误与追踪信息"""
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    def format_result(self, result: Any) -> dict:
        """为执行结果补充元数据（如时间戳）并统一为标准结构"""
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }


# AgentLanguage：语言适配层
# - 负责把（Goals/Actions/Memory）格式化为 LLM 需要的 Prompt
# - 负责从 LLM 的原始输出中解析出“要调用的工具与参数”
class AgentLanguage:
    def __init__(self):
        pass

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:
        raise NotImplementedError("Subclasses must implement this method")


    def parse_response(self, response: str) -> dict:
        raise NotImplementedError("Subclasses must implement this method")



# AgentFunctionCallingActionLanguage：基于“函数调用”范式的语言适配实现
# - 将 Goals 拼接为 system 消息
# - 将 Memory 规范化映射为 user/assistant 消息
# - 将 Actions 转换为符合 OpenAI 函数调用的 tools Schema
class AgentFunctionCallingActionLanguage(AgentLanguage):

    def __init__(self):
        super().__init__()

    def format_goals(self, goals: List[Goal]) -> List:
        # 把所有目标拼接为一个 system 消息，便于集中表达“要做什么/如何做”
        sep = "\n-------------------\n"
        goal_instructions = "\n\n".join([f"{goal.name}:{sep}{goal.description}{sep}" for goal in goals])
        return [
            {"role": "system", "content": goal_instructions}
        ]

    def format_memory(self, memory: Memory) -> List:
        """将 Memory 转换为对话消息格式，供 LLM 上下文使用"""
        # 记忆格式化策略：
        # - environment 的输出也作为 assistant 角色加入（让模型能“看到”工具执行结果）
        # - user/assistant 原样映射
        items = memory.get_memories()
        mapped_items = []
        for item in items:

            content = item.get("content", None)
            if not content:
                content = json.dumps(item, indent=4)

            if item["type"] == "assistant":
                mapped_items.append({"role": "assistant", "content": content})
            elif item["type"] == "environment":
                mapped_items.append({"role": "assistant", "content": content})
            else:
                mapped_items.append({"role": "user", "content": content})

        return mapped_items

    def format_actions(self, actions: List[Action]) -> [List,List]:
        """将已注册的动作转换为 OpenAI 函数调用所需的 tools Schema"""

        # 将注册的 Action 转为 OpenAI 函数调用工具的 Schema 数组
        tools = [
            {
                "type": "function",
                "function": {
                    "name": action.name,
                    # 描述过长可能无效，限制到 1024 字符
                    "description": action.description[:1024],
                    "parameters": action.parameters,
                },
            } for action in actions
        ]

        return tools

    def construct_prompt(self,
                         actions: List[Action],
                         environment: Environment,
                         goals: List[Goal],
                         memory: Memory) -> Prompt:

        # 构造最终 Prompt：Goals（system）+ Memory（历史消息）+ Tools（函数Schema）
        prompt = []
        prompt += self.format_goals(goals)
        prompt += self.format_memory(memory)

        tools = self.format_actions(actions)

        return Prompt(messages=prompt, tools=tools)

    def adapt_prompt_after_parsing_error(self,
                                         prompt: Prompt,
                                         response: str,
                                         traceback: str,
                                         error: Any,
                                         retries_left: int) -> Prompt:
        # 解析失败后的“自适应 Prompt”策略（此处保留扩展点，演示版不做修改）
        return prompt

    def parse_response(self, response: str) -> dict:
        """将 LLM 的响应解析为结构化格式（优先尝试 JSON 解析，失败则回退为终止工具）"""

        # 期望 LLM 返回 JSON 字符串：{"tool": 工具名, "args": {...}}
        try:
            return json.loads(response)

        except Exception as e:
            # 若无法解析，则将内容作为 message 交给终止工具，友好退出
            return {
                "tool": "terminate",
                "args": {"message":response}
            }


# Agent：智能体主循环
# - 维护并协调 G/A/M/E（目标/动作/记忆/环境）
# - 统一的 prompt 构造、响应解析、动作执行、记忆更新、终止判断
class Agent:
    def __init__(self,
                 goals: List[Goal],
                 agent_language: AgentLanguage,
                 action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str],
                 environment: Environment):
        """
        使用核心的 GAME 组件初始化智能体：
        - goals：目标与指令集合
        - agent_language：语言适配层（提示词构造与解析）
        - action_registry：动作注册表（可调用工具）
        - generate_response：LLM 调用函数
        - environment：动作执行环境
        """
        self.goals = goals
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goals: List[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """基于当前目标、记忆与动作集合构造提示词（Prompt）"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goals=goals,
            memory=memory
        )

    def get_action(self, response):
        # 解析 LLM 的返回，得到动作名与参数（invocation）
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        # 若当前选择的动作被标记为 terminal，则结束主循环
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        # 将用户输入写入记忆，作为本轮起始任务语境
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        使用“决策 + 执行结果”更新记忆：
        - 将助手的决策（response）作为 assistant 事件存入
        - 将环境执行结果（result）序列化为 JSON，作为 environment 事件存入
        """
        # 统一把“助手的决策（response）”与“环境执行结果（result）”写入记忆
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "environment", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        # 将 Prompt 发送给 LLM，得到“下一步动作/或文本回复”
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        执行该智能体的 GAME 主循环，可设置最大迭代次数：
        - 每轮：构造 Prompt -> 让 LLM 决策 -> 解析动作 -> 环境执行 -> 写回记忆 -> 终止判断
        """
        # 初始化记忆并写入用户任务
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # 1) 用当前 Goals/Actions/Memory 构造 Prompt
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # 2) 发送给 LLM，得到“将要调用的动作及其参数”或普通文本
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # 3) 解析动作与参数
            action, invocation = self.get_action(response)

            # 4) 在环境中真实执行动作
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # 5) 将“决策 + 结果”写回记忆，形成闭环
            self.update_memory(memory, response, result)

            # 6) 终止判断：如果动作为终止型，则跳出循环
            if self.should_terminate(response):
                break

        return memory


### OpenAI 函数调用（tools Schema）详解

在 Chat Completions API 中，通过 `tools` 字段向模型暴露可调用的函数（工具）。每个工具定义如下：

- `type`: 固定为 `function`
- `function`:
  - `name` (string): 工具名称（小写、下划线风格更稳妥，长度 ≤ 64 常见做法）
  - `description` (string): 工具用途的自然语言描述（有助于模型选择正确工具，建议简洁清晰）
  - `parameters` (object): 满足 JSON Schema Draft-07 的参数定义，用于指导模型正确组装入参
    - `type`: 通常为 `object`
    - `properties`: 各字段的类型与描述
    - `required`: 必填字段名列表
    - `additionalProperties`: 是否允许未声明字段（建议 `false` 以提高鲁棒性）

示例（与本笔记 `Action` 转换保持一致）：
```json
{
  "type": "function",
  "function": {
    "name": "read_project_file",
    "description": "Reads a file from the project.",
    "parameters": {
      "type": "object",
      "properties": {
        "name": { "type": "string", "description": "The file path to read" }
      },
      "required": ["name"],
      "additionalProperties": false
    }
  }
}
```

模型返回时会在 `message.tool_calls` 中给出调用的 `function.name` 与 `function.arguments`（JSON 字符串）。你的代码需：
- 解析 `arguments`（`json.loads`）
- 路由到本地实现函数执行
- 将执行结果写回对话历史（便于下一轮推理）

可选高级配置（按实际接口版本支持情况使用）：
- `tool_choice`: 强制使用某个工具或允许模型自由选择
- `parallel_tool_calls`: 是否允许并行调用（若可用）
- `response_format`: 强制 JSON 输出等（如需要结构化）

参考文档（官方）：
- OpenAI 工具/函数调用总览（Chat Completions）: [Function calling & tools](https://platform.openai.com/docs/guides/function-calling)
- JSON Schema 规范（参考）: [JSON Schema](https://json-schema.org/)
- Chat Completions 消息与工具调用字段说明: [Chat Completions API](https://platform.openai.com/docs/api-reference/chat)



In [4]:
# =============================== 示例：最小可运行 Agent ===============================
# 1) 定义智能体目标（Goals）：
#    - 读取项目中的每个文件
#    - 当已读取完毕时调用 terminate，并在消息中提供 README 的内容（示例环境如为空目录会直接终止）
goals = [
    Goal(priority=1, name="Gather Information", description="Read each file in the project"),
    Goal(priority=1, name="Terminate", description="Call the terminate call when you have read all the files "
                                                   "and provide the content of the README in the terminate message")
]

# 2) 指定语言适配器（基于函数调用的 Prompt/解析策略）
agent_language = AgentFunctionCallingActionLanguage()

# 3) 实现底层动作：读取文件
def read_project_file(name: str) -> str:
    with open(name, "r") as f:
        return f.read()

# 4) 实现底层动作：列出当前目录下的 .py 文件（最小示例）
def list_project_files() -> List[str]:
    return sorted([file for file in os.listdir(".") if file.endswith(".py")])


# 5) 注册动作：将 Python 函数“暴露”为可被 LLM 选择的工具
action_registry = ActionRegistry()
action_registry.register(Action(
    name="list_project_files",
    function=list_project_files,
    description="Lists all files in the project.",
    parameters={},
    terminal=False
))
action_registry.register(Action(
    name="read_project_file",
    function=read_project_file,
    description="Reads a file from the project.",
    parameters={
        "type": "object",
        "properties": {
            "name": {"type": "string"}
        },
        "required": ["name"]
    },
    terminal=False
))
action_registry.register(Action(
    name="terminate",
    function=lambda message: f"{message}\nTerminating...",
    description="Terminates the session and prints the message to the user.",
    parameters={
        "type": "object",
        "properties": {
            "message": {"type": "string"}
        },
        "required": []
    },
    terminal=True
))

# 6) 准备环境（负责真实执行动作并返回标准化结果）
environment = Environment()

# 7) 构建 Agent 实例（组装 G/A/M/E 与 LLM 响应函数）
agent = Agent(goals, agent_language, action_registry, generate_response, environment)

# 8) 运行智能体（输入一个自然语言任务），内部会进入循环直到触发终止或达到最大轮数
user_input = "Write a README for this project."
final_memory = agent.run(user_input)

# 9) 输出最终的记忆（包含用户任务、助手决策、环境执行结果等）
print(final_memory.get_memories())

Agent thinking...
Agent Decision: {"tool": "list_project_files", "args": {}}
Action Result: {'tool_executed': True, 'result': [], 'timestamp': '2025-10-30T03:23:57+0000'}
Agent thinking...
Agent Decision: {"tool": "terminate", "args": {"message": "It seems there are no files in the project directory, or there was an issue retrieving them. Since there is no content to base a README on, I'll need more information about the project to write an effective README file. Please provide details about the project such as its purpose, installation instructions, usage examples, and any other relevant information you would like to include."}}
Action Result: {'tool_executed': True, 'result': "It seems there are no files in the project directory, or there was an issue retrieving them. Since there is no content to base a README on, I'll need more information about the project to write an effective README file. Please provide details about the project such as its purpose, installation instructions, usa

## 项目结构与业务流程（Mermaid）

以下通过 Mermaid 图展示本项目的高层目录结构与 GAME 智能体业务流程。

### 项目结构概览
```mermaid
graph TD
  A[Agent_In_Action 根目录] --> B01[01-agent-llm-mcp]
  A --> B02[02-agent-multi-role]
  A --> B03[03-agent-build-docker-deploy]
  A --> B04[04-agent-evaluation]
  A --> B05[05-agent-model-finetuning]
  A --> B06[06-agent-model-inference]
  A --> B07[07-agent-llm_benchmark]

  B01 --> C011[ASimpleAgentFramework.ipynb]
  B01 --> C012[context-engineer/]
  B01 --> C013[mcp-demo/]

  B03 --> C031[backend/]
  B03 --> C032[frontend/]
  B03 --> C033[docker-compose.yml]

  C031 --> D031a[api_server.py]
  C031 --> D031b[agents/]
  C031 --> D031c[tools/]
  C031 --> D031d[config/]
  C031 --> D031e[data/]
  C031 --> D031f[utils/]
```

### GAME 智能体业务流程
```mermaid
sequenceDiagram
  participant U as 用户输入
  participant G as Goals / 指令
  participant M as Memory / 记忆
  participant L as AgentLanguage / 提示与解析
  participant A as Actions / 工具
  participant E as Environment / 环境执行
  participant LLM as 大语言模型

  U->>G: 任务/指令
  G->>L: 合并目标到 system 提示
  M->>L: 历史对话（user/assistant/environment）
  A->>L: 动作工具的 JSON Schema
  L->>LLM: 构造 Prompt + tools 并请求
  LLM-->>L: 工具调用/文本回复
  L->>A: 解析调用 {tool, args}
  A->>E: 在环境中执行对应函数
  E-->>M: 记录执行结果（标准化 + 时间戳）
  L->>M: 记录助手决策（response）
  M->>L: 新一轮上下文（若未终止）
  note over A: 若动作为终止型（terminal）则结束循环
```

